In [25]:
import os
import sys
sys.path.append("/test_cuda/")
import shutil
import math
import argparse
import torch
import torch.backends.cudnn as cudnn
import numpy as np
from data import cfg_mnet, cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
import cv2
from models.retinaface import RetinaFace
from utils.box_utils import decode, decode_landm
import time
import matplotlib.pyplot as plt
from torchvision.transforms.functional import crop, center_crop, rotate, InterpolationMode, pad, resize
#from torchvision.transforms.functional import rotate
#from torchvision.transforms.functional import InterpolationMode
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training, extract_face
from torch.utils.data import DataLoader, SubsetRandomSampler, SequentialSampler
from torchvision import datasets, transforms
from model import Backbone, Arcface, MobileFaceNet, Am_softmax, l2_norm
torch.cuda.empty_cache()

In [26]:
def check_keys(model, pretrained_state_dict):
    ckpt_keys = set(pretrained_state_dict.keys())
    model_keys = set(model.state_dict().keys())
    used_pretrained_keys = model_keys & ckpt_keys
    unused_pretrained_keys = ckpt_keys - model_keys
    missing_keys = model_keys - ckpt_keys
    #print('Missing keys:{}'.format(len(missing_keys)))
    #print('Unused checkpoint keys:{}'.format(len(unused_pretrained_keys)))
    #print('Used keys:{}'.format(len(used_pretrained_keys)))
    assert len(used_pretrained_keys) > 0, 'load NONE from pretrained checkpoint'
    return True

In [27]:
def remove_prefix(state_dict, prefix):
    ''' Old style model is stored with all names of parameters sharing common prefix 'module.' '''
    #print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}

In [28]:
def load_model(model, pretrained_path, load_to_cpu):
    print('Loading pretrained model from {}'.format(pretrained_path))
    if load_to_cpu:
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage)
    else:
        device = torch.cuda.current_device()
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage.cuda(device))
        print("Model loaded to GPU")
    if "state_dict" in pretrained_dict.keys():
        pretrained_dict = remove_prefix(pretrained_dict['state_dict'], 'module.')
    else:
        pretrained_dict = remove_prefix(pretrained_dict, 'module.')
    check_keys(model, pretrained_dict)
    model.load_state_dict(pretrained_dict, strict=False)
    return model

In [29]:
def detection_model(network="resnet50"):
    if network == "mobile0.25":
        cfg = cfg_mnet
        trained_model = "./weights/mobilenet0.25_Final.pth"
    elif network == "resnet50":
        cfg = cfg_re50
        trained_model = "./weights/Resnet50_Final.pth"
    # net and model
    net = RetinaFace(cfg=cfg, phase = 'test')
    net = load_model(net, trained_model, False)
    net.eval()
    cudnn.benchmark = True
    device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu") # Defines the computation device (cuda:0 => GPU)
    net = net.to(device)
    
    return net, cfg, device

In [30]:
def face_select(dets, selec_thresh, max_score=False):
    previous_area = 0
    max_area = 0
    prev_coords = np.zeros_like(dets[0])
    coords = np.zeros_like(dets[0])
    
    #print("face select: \n", dets, len(dets))
    if max_score == True: #Pre-crop that selects the face with higher score
        score = 0
        max_score = 0
        prev_crop_coords = np.zeros_like(dets[0])
        crop_coords = np.zeros_like(dets[0])

        for b in dets:
            if len(dets) == 1:
                crop_coords[:] = b
                
            if b[4]>score:
                score = b[4]
                prev_coords[:] = b
            else:
                max_score = score
                crop_coords[:] = prev_coords
        
        #crop_coords = list(map(int, crop_coords))
        #margin = 0.2 #x% around the bounding box
        #crop_height = crop_coords[3]-crop_coords[1] #ymax-ymin
        #crop_width = crop_coords[2]-crop_coords[0] #xmax-xZmin
        #crop_coords = (crop_coords[1]*(1-margin), crop_coords[0]*(1-margin), crop_height*(1+margin), crop_width*(1+margin))
        
        return crop_coords

    else:
        for b in dets:
            height = b[3]-b[1] #ymax-ymin
            width = b[2]-b[0] #xmax-xmin
            bbox_area = int(width)*int(height)


            #print(b[4])
            #if b[4]<0.001:
                #for i in range(len(coords)):
                    #coords[i] = prev_coords[i]
                #continue

            #b = list(map(int, b))
            for i in range(len(b)): # Turns into int every element from the detections except the score
                if i == 4:
                    continue
                else:
                    b[i] = int(b[i])

            #print("test", b, bbox_area, previous_area)              

            if len(dets) == 1: # Only one face present in the picture
                max_area = bbox_area
                for i in range(len(coords)):
                    coords[i] = b[i]
            else:
                if bbox_area > previous_area:
                    previous_area = bbox_area
                    for i in range(len(prev_coords)):
                        prev_coords[i] = b[i]
                else:
                    max_area = previous_area
                    for i in range(len(coords)):
                        coords[i] = prev_coords[i]


        face = np.append(coords, max_area)

        return coords

In [31]:
def crop_align(img, dets, selec_thresh, net, cfg, device, final_dir, save=False):
    '''
    b[0], b[1] is the top left corner of the bounding box
    b[2], b[3] is the lower right corner of the bounding box
    b[4] relates to the the score of the detection
    b[5], b[6] is the left eye
    b[7], b[8] is the right eye
    b[9], b[10] is the nose
    b[11], b[12] is the left of the mouth
    b[13], b[14] is the right of the mouth
    '''
    #print("dets 2 - crop align (pre): \n", dets)
    #img_raw = cv2.imread(img_path, cv2.IMREAD_COLOR)
    
    face_coords = face_select(dets, selec_thresh)
    face_coords = list(map(int, face_coords)) # Coordinates must be integers
    
    
    # -------------------- Rotation Stage ---------------------
    left_eye = (face_coords[5], face_coords[6]) # Components: (x, y)
    right_eye = (face_coords[7], face_coords[8])
    if left_eye[1] > right_eye[1]:               # Right eye is higher
        # Clock-wise rotation
        aux_point = (right_eye[0], left_eye[1])
        a = right_eye[0] - left_eye[0]
        b = right_eye[1] - aux_point[1]
        
        #cv2.circle(img_raw, left_eye, 10, (0, 255, 0), 4)
        #cv2.circle(img_raw, right_eye, 10, (0, 255, 0), 4)
        #cv2.circle(img_raw, aux_point, 10, (0, 255, 0), 4)
        
        #cv2.line(img_raw, left_eye, right_eye, (23, 23, 23), 2)
        #cv2.line(img_raw, aux_point, right_eye, (23, 23, 23), 2)
        #cv2.line(img_raw, left_eye, aux_point, (23, 23, 23), 2)
        #plt.imshow(cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)) 
        try:
            theta = np.rad2deg(np.arctan(b/a)) # Angle of rotation in degrees
            rotated_tensor = rotate(img.squeeze(), angle=theta, interpolation=InterpolationMode.BILINEAR, center=right_eye)
        except ZeroDivisionError:
            rotated_tensor = img.squeeze()
            
        #print("Right eye is higher, therefore, a clock-wise rotation of {} is applied".format(-theta))
        

    else:                                        # Left eye is higher
        # Counter clock-wise rotation
        aux_point = (left_eye[0], right_eye[1])
        a = right_eye[0] - left_eye[0]
        b = left_eye[1] - aux_point[1]
        
        #cv2.circle(img_raw, left_eye, 10, (0, 255, 0), 4)
        #cv2.circle(img_raw, right_eye, 10, (0, 255, 0), 4)
        #cv2.circle(img_raw, aux_point, 10, (0, 255, 0), 4)
        
        #plt.imshow(img_raw)
        try:
            theta = np.rad2deg(np.arctan(b/a))
            #print("Left eye is higher, therefore, a clock-wise rotation of {} degrees is applied".format(-theta))
            rotated_tensor = rotate(img.squeeze(), angle=-theta, interpolation=InterpolationMode.BILINEAR, center=left_eye)
        except ZeroDivisionError:
            rotated_tensor = img.squeeze()
    
    #margin = 0.5 #x% around the bounding box
    #crop_height = face_coords[3]-face_coords[1] #ymax-ymin
    #crop_width = face_coords[2]-face_coords[0] #xmax-xZmin
    #crop_coords = (face_coords[1]*(1-margin), face_coords[0]*(1-margin), crop_height*(1+margin), crop_width*(1+margin))
    #crop_coords = list(map(int, crop_coords))
    #rotated_tensor = crop(rotated_tensor, *crop_coords)
    #plt.imshow(rotated_tensor.squeeze().permute(1, 2, 0).cpu().numpy().astype(int))
    
    # -------------------- New Bounding Box computing ---------------------
    # The image is rotated, a new bbox must be generated. 
    
    # TBD: optimization by performing a preliminary crop in order to try and isolate only the relevant face
    
    loc, conf, _ = net(rotated_tensor.unsqueeze(0))  # Forward pass that gives the results <--------------
    
    im_height = rotated_tensor.shape[1]
    im_width = rotated_tensor.shape[2]
    
    resize1 = 1
    new_scale = torch.Tensor([rotated_tensor.shape[2], rotated_tensor.shape[1], rotated_tensor.shape[2], rotated_tensor.shape[1]])
    new_scale = new_scale.to(device)
    
    new_priorbox = PriorBox(cfg, image_size=(im_height, im_width))
    new_priors = new_priorbox.forward()
    new_priors = new_priors.to(device)
    new_prior_data = new_priors.data
    
    new_boxes = decode(loc.data.squeeze(0), new_prior_data, cfg['variance'])
    new_boxes = new_boxes * new_scale / resize1
    new_boxes = new_boxes.cpu().numpy() # Tensor is moved to CPU (numpy doesn't support GPU)
    new_scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
    

    # Score's threshold
    confidence_threshold = 0.00002 # Default value
    inds = np.where(new_scores > confidence_threshold)[0]
    new_boxes = new_boxes[inds]
    new_scores = new_scores[inds]

    # keep top-K before NMS
    top_k = 500 # Default value
    order = new_scores.argsort()[::-1][:top_k] # Extracts the indexes relating to the top scores
    new_boxes = new_boxes[order] # Array [300, 4] where in each line are the coordinates
    new_scores = new_scores[order] # Array [1, 300]
    
    
    # do NMS
    nms_threshold = 0.00002 # Default value
    new_dets = np.hstack((new_boxes, new_scores[:, np.newaxis])).astype(np.float32, copy=False)
    keep = py_cpu_nms(new_dets, nms_threshold)
    new_dets = new_dets[keep, :]
    
    #print("dets 3 - crop align (new dets): \n", new_dets)
    
    
    # keep top-K faster NMS
    #keep_top_k = 500 # Default value
    #new_dets = new_dets[:keep_top_k, :]
    
    #rotated_bbox = new_dets[0]
    rotated_bbox = face_select(new_dets, selec_thresh)
    #print("rotated_bbox 1", rotated_bbox)
    rotated_bbox = list(map(int, rotated_bbox))
    #print("rotated_bbox 2", rotated_bbox)
    

    
    
    # -------------------- Cropping Stage ---------------------
    crop_height = int((rotated_bbox[3]-rotated_bbox[1])*1.2) #ymax-ymin
    crop_width = int((rotated_bbox[2]-rotated_bbox[0])*1.2) #xmax-xZmin
    crop_coordinates = (rotated_bbox[1], rotated_bbox[0], crop_height, crop_width)
    cropped_tensor = crop(rotated_tensor, *crop_coordinates)
    
    #plt.imshow(cropped_tensor.squeeze().permute(1, 2, 0).cpu().numpy().astype(int))
    
    
    if save == True:
        #image_array = cropped_tensor.permute(1,2,0).cpu().numpy()
        
        # Define the desired output size
        #desired_size = (180, 180)  # (height, width)

        # Calculate the padding values
        #height_diff = desired_size[0] - cropped_tensor.shape[1]
        #width_diff = desired_size[1] - cropped_tensor.shape[2]
        #top_pad = height_diff // 2
        #bottom_pad = height_diff - top_pad
        #left_pad = width_diff // 2
        #right_pad = width_diff - left_pad

        # Apply padding to the tensor
        #padded_tensor = pad(cropped_tensor, (left_pad, right_pad, top_pad, bottom_pad))
        
        final_size = (160, 160)
        #resized_tensor = resize(padded_tensor, final_size)
        resized_tensor = resize(cropped_tensor, final_size)
        
        image_array = resized_tensor.permute(1,2,0).cpu().numpy()
        
        # Convert the numpy array to BGR format (required by OpenCV)
        cropped_image = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
        
        cv2.imwrite(final_dir, cropped_image)
    return cropped_tensor

In [32]:
# https://github.com/biubug6/Pytorch_Retinaface/
def face_detection(net, cfg, device, img, final_dir, img_raw, save=False):
    #save_image = False
    torch.set_grad_enabled(False)
    
    resize1 = 1
    
    im_height, im_width, _ = img.shape
    scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])

    img = img.transpose(2, 0, 1)
    img = torch.from_numpy(img).unsqueeze(0)
    img = img.to(device)
    scale = scale.to(device)
    # Testing stage
    
    tic = time.time()
    loc, conf, landms = net(img)  # Forward pass that gives the results <--------------
    #print('Forward time: {:.4f}'.format(time.time() - tic))
        
    priorbox = PriorBox(cfg, image_size=(im_height, im_width))
    priors = priorbox.forward()
    priors = priors.to(device)
    prior_data = priors.data
    boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
    
    boxes = boxes * scale / resize1
    boxes = boxes.cpu().numpy() # Tensor is moved to CPU (numpy doesn't support GPU)
    scores = conf.squeeze(0).data.cpu().numpy()[:,1]
    landms = decode_landm(landms.data.squeeze(0), prior_data, cfg['variance'])
    scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                            img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                            img.shape[3], img.shape[2]])
    scale1 = scale1.to(device)
    landms = landms * scale1 / resize1
    landms = landms.cpu().numpy()

    # Score's threshold
    confidence_threshold = 0.00002 # Default value
    inds = np.where(scores > confidence_threshold)[0]
    boxes = boxes[inds]
    landms = landms[inds]
    scores = scores[inds]

    # keep top-K before NMS
    top_k = 500 # Default value
    order = scores.argsort()[::-1][:top_k] # Extracts the indexes relating to the top scores
    boxes = boxes[order] # Array [300, 4] where in each line are the coordinates
    landms = landms[order] # Array [300, 10]
    scores = scores[order] # Array [1, 300]
    
    print(max(scores))
    
    # do NMS
    nms_threshold = 0.00002 # Default value
    dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
    keep = py_cpu_nms(dets, nms_threshold)
    dets = dets[keep, :]
    landms = landms[keep]

    # keep top-K faster NMS
    keep_top_k = 750 # Default value
    dets = dets[:keep_top_k, :]
    landms = landms[:keep_top_k, :]
    

    dets = np.concatenate((dets, landms), axis=1)
    #print(dets)
    #print("dets 1 - face selection: \n", dets)
    
    #for b in dets:
        #text = "{:.8f}".format(b[4])
        #b = list(map(int, b))
        #cv2.rectangle(img_raw, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 2)
        #cx = b[0]
        #cy = b[1] + 12
        #cv2.circle(img_raw, (0, 0), 10, (0, 255, 0), 4)
        #cv2.circle(img_raw, (b[0], b[1]), 1, (255, 0, 255), 4)
        #cv2.circle(img_raw, (b[2], b[3]), 1, (255, 0, 255), 4)
        #cv2.putText(img_raw, text, (cx, cy),
                    #cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))

        # landms
        #cv2.circle(img_raw, (b[5], b[6]), 1, (0, 0, 255), 4)
        #cv2.circle(img_raw, (b[7], b[8]), 1, (0, 255, 255), 4)
        #cv2.circle(img_raw, (b[9], b[10]), 1, (255, 0, 255), 4)
        #cv2.circle(img_raw, (b[11], b[12]), 1, (0, 255, 0), 4)
        #cv2.circle(img_raw, (b[13], b[14]), 1, (255, 0, 0), 4)


        #plt.imshow(img_raw)
    
    #Pre-crop
    #pre_crop_coords = face_select(dets, 0.1, pre_crop=True)
    #pre_crop_coords = list(map(int, pre_crop_coords))
    #pre_img = crop(img, *pre_crop_coords)
    
    cropped = crop_align(img, dets, 0.1, net, cfg, device, final_dir, save)
    

    return cropped, dets

In [33]:
def c_crop(image, margin_percentage):
    height, width = image.shape[:2]
    
    # Calculate the margin size in pixels
    margin_height = int(height * margin_percentage / 100)
    margin_width = int(width * margin_percentage / 100)

    # Calculate the starting coordinates for the crop
    start_y = margin_height
    start_x = margin_width

    # Calculate the target height and width after cropping
    target_height = height - 2 * margin_height
    target_width = width - 2 * margin_width

    # Perform the center crop with margin
    cropped_image = image[start_y:start_y+target_height, start_x:start_x+target_width]

    return cropped_image

In [11]:
#raw_root = "/test_cuda/cplfw_raw"
#
#for file in os.listdir(raw_root):
#    file_path = os.path.join(raw_root, file)
#    identity = ""
#    for i in range(len(file)-6):
#        identity += str(file[i])
#        
#    id_path = os.path.join(root, identity)
#    if os.path.exists(id_path):
#        print("Directory already exists")
#        shutil.move(file_path, id_path)
#    else:
#        os.makedirs(id_path, exist_ok=True)
#        shutil.move(file_path, id_path)

In [36]:
file_list = os.listdir("/test_cuda/datasets/cplfw/aligned_images/")
dest = "/test_cuda/datasets/CPLFW/cplfw_cropped3/"
for file_name in file_list:
        # Extract the folder name (e.g., 'aaa_aaa') from the file name
        folder_name = '_'.join(file_name.split("_")[:-1])
        # Create the folder if it doesn't exist
    
        path = os.path.join(dest, folder_name)
        os.makedirs(path, exist_ok=True)
        
        #print(folder_name, file_name)
        #origin = os.path.join("/test_cuda/datasets/cplfw/aligned_images/", file_name)
        # Move the file to the respective folder
        #shutil.move(origin, os.path.join(dest, file_name))

In [43]:
print(len(os.listdir("/test_cuda/datasets/XQLFW/xqlfw_cropped")))
#print(len(np.load("/test_cuda/InsightFace_Pytorch-master/data/faces_emore/cplfw_list.npy")))

5749


In [107]:
# Testing folder
root = "/test_cuda/datasets/CPLFW/cplfw"
dest_dir = "/test_cuda/datasets/cplfw/aligned images/"
pairs_dir = "/test_cuda/datasets/CPLFW/cplfw_pairs.txt"

In [108]:
#t_tic = time.time()
#
#net, cfg, device = detection_model()
#
#while True:
#    if os.path.exists(dest_dir) and os.path.isdir(dest_dir):
#        break
#    else:
#        os.makedirs(dest_dir, exist_ok=True)
#                
#for i in os.listdir(root):
#    id_path = os.path.join(root, i)
#    
#    cropped_dir = os.path.join(dest_dir, i)
#    os.makedirs(cropped_dir, exist_ok=True)
#    for files in os.listdir(id_path):
#        if not files.startswith('.'):
#            file_path = os.path.join(id_path, files)
#            final_dir = os.path.join(cropped_dir, files)
#            
#            print(file_path)
#            img_raw = cv2.imread(file_path, cv2.IMREAD_COLOR)
#            cropped_img_raw = c_crop(img_raw, 22)
#            img_raw_rgb = cv2.cvtColor(cropped_img_raw, cv2.COLOR_BGR2RGB)
#
#            img = np.float32(img_raw_rgb)
#            _, dets = face_detection(net, cfg, device, img, final_dir, img_raw, save=True)
#                        
#print('Total time: {:.4f}'.format(time.time() - t_tic)) 

In [109]:
torch.cuda.empty_cache()

workers = 16
batch_size = 100

transf = transforms.Compose([
    transforms.Resize((112, 112)),
    np.float32,
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[torch.LongTensor([2, 1, 0])]),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    #fixed_image_standardization #Normalizes tensors to [-1, 1]
])

dataset = datasets.ImageFolder(dest_dir, transform=transf)

data_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SequentialSampler(dataset)
)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

FileNotFoundError: Couldn't find any class folder in /test_cuda/datasets/cplfw/aligned images/.

In [106]:
print(len(dataset.class_to_idx))
print(len(os.listdir("/test_cuda/datasets/XQLFW/xqlfw_aligned_112/")))

3931
5749


In [78]:
backbone = MobileFaceNet(512).to(device)
print('MobileFaceNet model generated')

MobileFaceNet model generated


In [79]:
backbone.load_state_dict(torch.load("/test_cuda/InsightFace_Pytorch-master/work_space/save/model_mobilefacenet.pth"))

<All keys matched successfully>

In [80]:
## Load pretrained resnet model
#resnet = InceptionResnetV1(
#    classify=False,
#    pretrained='casia-webface'
#).to(device)

In [81]:
#resnet = InceptionResnetV1(num_classes=10000).to(device)
#
#dic = torch.load('/test_cuda/models/train_49.pth')
##dic = torch.load('/test_cuda/model.pth')
#keys_to_remove = ['logits.weight', 'logits.bias']
#for key in keys_to_remove:
#    dic.pop(key, None)
#    
#resnet.load_state_dict(dic)

In [92]:
classes = []
embeddings = []
backbone.eval()
with torch.no_grad():
    for xb, yb in data_loader:
        xb = xb.to(device)
        b_embeddings = backbone(xb)
        b_embeddings = b_embeddings.to('cpu').numpy()
        classes.extend(yb.numpy())
        embeddings.append(b_embeddings)

In [94]:
print(len(embeddings))

117


In [48]:
crop_path = []
for i in os.listdir('/test_cuda/datasets/CPLFW/cplfw_cropped'):
    cropped = os.path.join('/test_cuda/datasets/CPLFW/cplfw_cropped', i)
    for j in os.listdir(cropped):
        final = os.path.join(cropped, j)    
        crop_path.append(final)

crop_path = sorted(crop_path)

In [96]:
print(len(crop_path))

11656


In [49]:
embeddings_dict = dict(zip(crop_path, embeddings))

In [50]:
from sklearn.model_selection import KFold
from scipy import interpolate

# LFW functions taken from David Sandberg's FaceNet implementation
def distance(embeddings1, embeddings2, distance_metric=0):
    if distance_metric==0:
        # Euclidian distance
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff),1)
    elif distance_metric==1:
        # Distance based on cosine similarity
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=1)
        norm = np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1)
        similarity = dot / norm
        dist = np.arccos(similarity) / math.pi
    else:
        raise 'Undefined distance metric %d' % distance_metric

    return dist

def calculate_roc(thresholds, embeddings1, embeddings2, actual_issame, nrof_folds=10, distance_metric=0, subtract_mean=False):
    assert(embeddings1.shape[0] == embeddings2.shape[0])
    assert(embeddings1.shape[1] == embeddings2.shape[1])
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    nrof_thresholds = len(thresholds)
    k_fold = KFold(n_splits=nrof_folds, shuffle=True)

    tprs = np.zeros((nrof_folds,nrof_thresholds))
    fprs = np.zeros((nrof_folds,nrof_thresholds))
    accuracy = np.zeros((nrof_folds))

    is_false_positive = []
    is_false_negative = []

    indices = np.arange(nrof_pairs)

    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        if subtract_mean:
            mean = np.mean(np.concatenate([embeddings1[train_set], embeddings2[train_set]]), axis=0)
        else:
            mean = 0.0
        dist = distance(embeddings1-mean, embeddings2-mean, distance_metric)

        # Find the best threshold for the fold
        acc_train = np.zeros((nrof_thresholds))
        for threshold_idx, threshold in enumerate(thresholds):
            _, _, acc_train[threshold_idx], _ ,_ = calculate_accuracy(threshold, dist[train_set], actual_issame[train_set])
        best_threshold_index = np.argmax(acc_train)
        for threshold_idx, threshold in enumerate(thresholds):
            tprs[fold_idx,threshold_idx], fprs[fold_idx,threshold_idx], _, _, _ = calculate_accuracy(threshold, dist[test_set], actual_issame[test_set])
        _, _, accuracy[fold_idx], is_fp, is_fn = calculate_accuracy(thresholds[best_threshold_index], dist[test_set], actual_issame[test_set])

        tpr = np.mean(tprs,0)
        fpr = np.mean(fprs,0)
        is_false_positive.extend(is_fp)
        is_false_negative.extend(is_fn)

    return tpr, fpr, accuracy, is_false_positive, is_false_negative

def calculate_accuracy(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    tp = np.sum(np.logical_and(predict_issame, actual_issame))
    fp = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    tn = np.sum(np.logical_and(np.logical_not(predict_issame), np.logical_not(actual_issame)))
    fn = np.sum(np.logical_and(np.logical_not(predict_issame), actual_issame))

    is_fp = np.logical_and(predict_issame, np.logical_not(actual_issame))
    is_fn = np.logical_and(np.logical_not(predict_issame), actual_issame)

    tpr = 0 if (tp+fn==0) else float(tp) / float(tp+fn)
    fpr = 0 if (fp+tn==0) else float(fp) / float(fp+tn)
    acc = float(tp+tn)/dist.size
    return tpr, fpr, acc, is_fp, is_fn

def calculate_val(thresholds, embeddings1, embeddings2, actual_issame, far_target, nrof_folds=10, distance_metric=0, subtract_mean=False):
    assert(embeddings1.shape[0] == embeddings2.shape[0])
    assert(embeddings1.shape[1] == embeddings2.shape[1])
    nrof_pairs = min(len(actual_issame), embeddings1.shape[0])
    nrof_thresholds = len(thresholds)
    k_fold = KFold(n_splits=nrof_folds, shuffle=True)

    val = np.zeros(nrof_folds)
    far = np.zeros(nrof_folds)

    indices = np.arange(nrof_pairs)

    for fold_idx, (train_set, test_set) in enumerate(k_fold.split(indices)):
        if subtract_mean:
            mean = np.mean(np.concatenate([embeddings1[train_set], embeddings2[train_set]]), axis=0)
        else:
            mean = 0.0
        dist = distance(embeddings1-mean, embeddings2-mean, distance_metric)

        # Find the threshold that gives FAR = far_target
        far_train = np.zeros(nrof_thresholds)
        for threshold_idx, threshold in enumerate(thresholds):
            _, far_train[threshold_idx] = calculate_val_far(threshold, dist[train_set], actual_issame[train_set])
        if np.max(far_train)>=far_target:
            f = interpolate.interp1d(far_train, thresholds, kind='slinear')
            threshold = f(far_target)
        else:
            threshold = 0.0

        val[fold_idx], far[fold_idx] = calculate_val_far(threshold, dist[test_set], actual_issame[test_set])

    val_mean = np.mean(val)
    far_mean = np.mean(far)
    val_std = np.std(val)
    return val_mean, val_std, far_mean

def calculate_val_far(threshold, dist, actual_issame):
    predict_issame = np.less(dist, threshold)
    true_accept = np.sum(np.logical_and(predict_issame, actual_issame))
    false_accept = np.sum(np.logical_and(predict_issame, np.logical_not(actual_issame)))
    
    n_same = np.sum(actual_issame)
    n_diff = np.sum(np.logical_not(actual_issame))
    val = float(true_accept) / float(n_same)
    far = float(false_accept) / float(n_diff)
        
    return val, far


def evaluate(embeddings, actual_issame, nrof_folds=10, distance_metric=0, subtract_mean=False):
    # Calculate evaluation metrics
    thresholds = np.arange(0, 4, 0.01)
    embeddings1 = embeddings[0::2]
    embeddings2 = embeddings[1::2]
    tpr, fpr, accuracy, fp, fn  = calculate_roc(thresholds, embeddings1, embeddings2,
        np.asarray(actual_issame), nrof_folds=nrof_folds, distance_metric=distance_metric, subtract_mean=subtract_mean)
    thresholds = np.arange(0, 4, 0.001)
    val, val_std, far = calculate_val(thresholds, embeddings1, embeddings2,
        np.asarray(actual_issame), 1e-3, nrof_folds=nrof_folds, distance_metric=distance_metric, subtract_mean=subtract_mean)
    return tpr, fpr, accuracy, val, val_std, far, fp, fn

def add_extension(path):
    if os.path.exists(path+'.jpg'):
        return path+'.jpg'
    elif os.path.exists(path+'.png'):
        return path+'.png'
    else:
        raise RuntimeError('No file "%s" with extension png or jpg.' % path)

def get_paths(lfw_dir, pairs):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []

    for index in range(0, len(pairs)-1):
        if pairs[index][1] == "1" and pairs[index+1][1] == "1":
            if index % 2 == 0:
                identity0 = ''.join([char for char in pairs[index][0][:-6]])
                identity1 = ''.join([char for char in pairs[index+1][0][:-6]])
                
                path0 = os.path.join(lfw_dir, identity0, pairs[index][0])
                path1 = os.path.join(lfw_dir, identity1, pairs[index+1][0])
                
                #print("True --->", index, pairs[index][0], pairs[index+1][0])
                issame = True
            else:
                continue          
        elif pairs[index][1] == "0" or pairs[index+1][1] == "0":
            if index % 2 == 0:
                identity0 = ''.join([char for char in pairs[index][0][:-6]])
                identity1 = ''.join([char for char in pairs[index+1][0][:-6]])

                path0 = os.path.join(lfw_dir, identity0, pairs[index][0])
                path1 = os.path.join(lfw_dir, identity1, pairs[index+1][0])
                
                #print("False --->", index, pairs[index][0], pairs[index+1][0])
                issame = False
            else:
                continue
        if os.path.exists(path0) and os.path.exists(path1):    # Only add the pair if both paths exist
            path_list += (path0,path1)
            issame_list.append(issame)
        else:
            nrof_skipped_pairs += 1
    if nrof_skipped_pairs>0:
        print('Skipped %d image pairs' % nrof_skipped_pairs)

    return path_list, issame_list

def read_pairs(pairs_filename):
    pairs = []
    with open(pairs_filename, 'r') as f:
        for line in f.readlines()[0:]:
            pair = line.strip().split()
            pairs.append(pair)
    return np.array(pairs, dtype=object)

In [51]:
pairs = read_pairs(pairs_dir)
path_list, issame_list = get_paths(dest_dir, pairs)

embeddings = np.array([embeddings_dict[path] for path in path_list])
tpr, fpr, accuracy, val, val_std, far, fp, fn = evaluate(embeddings, issame_list)

KeyError: '/test_cuda/datasets/CPLFW/cplfw_cropped/Ingrid_Betancourt/Ingrid_Betancourt_1.jpg'

In [23]:
#print("tpr: \n", tpr)
#print(" ")
#print("fpr: \n", fpr)
#print(" ")
#print("accuracy: \n", accuracy)
#print(" ")
#print("val: \n", val)
#print(" ")
#print("val_std: \n", val_std)
#print(" ")
#print("far: \n", far)
#print(" ")
#print("fp: \n", fp)
#print(" ")
#print("fn: \n", fn)
print(" ")
np.mean(accuracy) #0.8323333333333334 before finetuning -------------- 0.7896666666666666 casia-webface
#0.5826666666666667
#0.5175000000000001 after training
#----------
#0.7216666666666667 trained for 10 epochs and unfrozen from mixed_7a forward
#0.7230000000000001 trained for 10 epochs and unfrozen from repeat 3 forward
#0.7186666666666668 trained for 10 epochs and unfrozen from block 8 forward
#0.7443333333333333 trained for 10 epochs and unfrozen from avgpool_1a forward
#0.7436666666666667 trained for 10 epochs and unfrozen from dropout forward
#0.7446666666666666 trained for 10 epochs and unfrozen from last_linear forward
#0.7513333333333333 trained for 10 epochs and unfrozen from last_bn forward
#0.7558333333333335 trained for 10 epochs and everything is frozen except the classifier
#0.757 trained for 10 epochs, everything is frozen except the classifier and m reduce from 0.001 to 0.00001
#0.7511666666666666 trained for 10 epochs, everything is frozen except the classifier and m inreased from 0.00001 to 0.5
#0.7528333333333334 trained for 10 epochs, everything is frozen except the classifier, s=6 and m = 0.5  (very high training loss)
#0.7548333333333334 trained for 10 epochs, everything is frozen except the classifier, s=6 and m = 1 
#0.7573333333333334 trained for 10 epochs, everything is frozen except the classifier, s=64 and m = 0.5
#0.7523333333333333 cross entropy added to optimizer, s=64 and m=0.5 
#0.7545 weight decay from 1e-4 to 5e-4 and optimizer to the power of 3
#0.7543333333333334 same as before but m=0.4
#0.7545 same as before but m=0.3
#0.756 5 epochs, s=64 and m=0.4
#0.7555 5 epochs, s=64 and m=0.3
#0.7526666666666666 5 epochs, s=64 and m=0.2
#0.7503333333333333 5 epochs, s=64 and m=0.1
#0.7526666666666668 5 epochs, s=64 and m=0.01
#0.7511666666666668 256 batch size
#0.7505 64 batch size
#0.7500000000000001 lr 0.25 5 epochs s 64 m 0.01
#0.7528333333333332 lr 0.25 10 epochs s 64 m 0.01
#0.7528333333333332 step optimizer 
#0.7464999999999999 first training with casia-webface as base
#0.6675 trained for 5 epochs and unfrozen from mixed_7a forward
#0.6685000000000001 lr 0.1 s=64, m=0.01
#0.6623333333333334 last model retrainned through the whole network
#0.7409999999999999 10 epochs
#0.7415
#0.6318333333333334
#0.6495
#0.7420000000000001 everything frozen
#0.744 lr 0.001 s=30 m=0.01
#0.7431666666666666 same as before but trained for only 2 epochs
#0.7431666666666666 3 epochs m=0.001
#0.7458333333333333 same as before but m=0.5

0.5311666666666666

In [24]:
import sys
import torchvision
print("Pytorch: ", torch.__version__)
print("Torchvision: ", torchvision.__version__)
print("CUDA:", torch.version.cuda)
print("Python:", sys.version)
print("CuDNN:", torch.backends.cudnn.version())

Pytorch:  1.14.0a0+410ce96
Torchvision:  0.15.0a0
CUDA: 11.8
Python: 3.8.10 (default, May 26 2023, 14:05:08) 
[GCC 9.4.0]
CuDNN: 8700
